*** TO DO FOR CATAN: ***
RAINBOW: 
    1. vs Random
    2. vs Weighted Random
    3. vs MTCS
    4. vs Victory Point
    5. vs AlphaBeta
Masked PPO the same 
NFSP 
MuZero

In [ ]:
import sys

# sys.path.append("/content/rl-research")
sys.path.append("../..")
import gymnasium as gym
import torch

from wrappers import CatanatronWrapper
from utils import CategoricalCrossentropyLoss, KLDivergenceLoss


from dqn.rainbow.rainbow_agent import RainbowAgent
from agent_configs import RainbowConfig
from game_configs.catan_config import SinglePlayerCatanConfig
from catanatron import Game, RandomPlayer, Color
from catanatron.players.mcts import MCTSPlayer
from catanatron.players.minimax import AlphaBetaPlayer
from catanatron.players.playouts import GreedyPlayoutsPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.value import ValueFunctionPlayer

config_dict = {
    "dense_layer_widths": [256, 256, 256, 256],
    "value_hidden_layers_widths": [],  #
    "advatage_hidden_layers_widths": [],  #
    "adam_epsilon": 3.125e-4,
    "learning_rate": 0.001,
    "training_steps": 30000,
    "per_epsilon": 1e-6,  #
    "per_alpha": 0.5,
    "per_beta": 0.5,  # For RIAYN should be no annealing # 0.4
    "minibatch_size": 32,
    "replay_buffer_size": 500000,
    "min_replay_buffer_size": 5000,
    "transfer_interval": 100,
    "n_step": 9,
    "kernel_initializer": "orthogonal",  #
    "loss_function": KLDivergenceLoss(),  # KLDivergence()
    "clipnorm": 0.0,  # 2.0
    "discount_factor": 0.99,
    "atom_size": 51,
    "replay_interval": 512,
}
game_config = SinglePlayerCatanConfig()
config = RainbowConfig(config_dict, game_config)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
import catanatron.gym
import gymnasium as gym

env = CatanatronWrapper(
    gym.make(
        "catanatron/Catanatron-v0",
        config={
            "enemies": [RandomPlayer(Color.RED)],
            "invalid_action_reward": -10,
            "map_type": "BASE",
            "vps_to_win": 10,
            "representation": "vector",
        },
    )
)
agent = RainbowAgent(env, config, "rainbow-catan-3vps", device)
agent.checkpoint_interval = 10
agent.test_interval = 50
agent.test_trials = 25
agent.train()

Using default save_intermediate_weights     : False
Using         training_steps                : 30000
Using         adam_epsilon                  : 0.0003125
Using default momentum                      : 0.9
Using         learning_rate                 : 0.001
Using         clipnorm                      : 0.0
Using default optimizer                     : <class 'torch.optim.adam.Adam'>
Using default weight_decay                  : 0.0
Using         loss_function                 : <utils.utils.KLDivergenceLoss object at 0x128a5d6a0>
Using default activation                    : relu
Using         kernel_initializer            : orthogonal
Using         minibatch_size                : 64
Using         replay_buffer_size            : 500000
Using         min_replay_buffer_size        : 5000
Using default num_minibatches               : 1
Using default training_iterations           : 1
Using default print_interval                : 100
RainbowConfig
Using default residual_layers           

KeyboardInterrupt: 

In [ ]:
# shared network but not shared buffer?
# 1 vs 2 minibatches

from dqn.NFSP.nfsp_agent_clean import NFSPDQN
from agent_configs import NFSPDQNConfig
from game_configs import CatanConfig
from utils import KLDivergenceLoss, CategoricalCrossentropyLoss, HuberLoss, MSELoss
from torch.optim import Adam, SGD

config_dict = {
    "shared_networks_and_buffers": False,
    "training_steps": 50000,
    "anticipatory_param": 0.1,
    "replay_interval": 128,  #
    "num_minibatches": 1,  # or 2, could be 2 minibatches per network, or 2 minibatches (1 for each network/player)
    "learning_rate": 0.1,
    "momentum": 0.0,
    "optimizer": SGD,
    "loss_function": MSELoss(),
    "min_replay_buffer_size": 1000,
    "minibatch_size": 128,
    "replay_buffer_size": 1e5,
    "transfer_interval": 300,
    "residual_layers": [],
    "conv_layers": [],
    "dense_layer_widths": [128],
    "value_hidden_layer_widths": [],
    "advantage_hidden_layer_widths": [],
    "noisy_sigma": 0.0,
    "eg_epsilon": 0.06,
    # "eg_epsilon_final": 0.06,
    "eg_epsilon_decay_type": "inverse_sqrt",
    "eg_epsilon_decay_final_step": 0,
    "sl_learning_rate": 0.005,
    "sl_momentum": 0.0,
    # "sl_weight_decay": 1e-9,
    # "sl_clipnorm": 1.0,
    "sl_optimizer": SGD,
    "sl_loss_function": CategoricalCrossentropyLoss(),
    "sl_min_replay_buffer_size": 1000,
    "sl_minibatch_size": 128,
    "sl_replay_buffer_size": 100000,
    "sl_residual_layers": [],
    "sl_conv_layers": [],
    "sl_dense_layer_widths": [128],
    "sl_clip_low_prob": 0.0,
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "per_epsilon": 0.00001,
    "n_step": 1,
    "atom_size": 1,
    "dueling": False,
    "clipnorm": 10.0,
    "sl_clipnorm": 10.0,
}
config = NFSPDQNConfig(
    config_dict=config_dict,
    game_config=CatanConfig(),
)
config.save_intermediate_weights = False
from custom_gym_envs.envs.catan import (
    env as catan_env,
    CatanAECEnv,
)

env = catan_env(
    num_players=2,
    map_type="BASE",
    vps_to_win=10,
    representation="vector",
    invalid_action_reward=-10,
)

env = ActionMaskInInfoWrapper(env)
# env = FrameStackWrapper(env, 4, channel_first=False)

agent = NFSPDQN(env, config, name="nfsp-catan", device="cuda:0")
agent.checkpoint_interval = 100
agent.test_interval = 500
agent.test_trials = 10
agent.train()

In [1]:
import sys
import catanatron.gym

sys.path.append("../../")
import gymnasium as gym
import random

env = gym.make("catanatron/Catanatron-v0")
observation, info = env.reset()
for _ in range(1000):
    # your agent here (this takes random actions)
    action = random.choice(info["valid_actions"])

    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    if done:
        observation, info = env.reset()
env.close()

In [4]:
from catanatron import Game, RandomPlayer, Color
from catanatron.players.mcts import MCTSPlayer
from catanatron.players.minimax import AlphaBetaPlayer
from catanatron.players.playouts import GreedyPlayoutsPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.value import ValueFunctionPlayer

# Instantiate two random players
player1 = RandomPlayer(Color.RED)
player2 = RandomPlayer(Color.BLUE)

# Create a 2-player game (you can fill remaining slots with random agents if needed)
players = [player1, player2]
game = Game(players)

winner = game.play()
print(f"Winner: {winner}")


def play_game(player1, player2):
    player1 = player1(Color.RED)
    player2 = player2(Color.BLUE)
    game = Game([player1, player2])
    winner = game.play()

    if winner == Color.RED:
        return 1
    elif winner == Color.BLUE:
        return -1
    else:
        return 0

Winner: Color.BLUE


In [ ]:
import pandas as pd
import random
from tqdm import tqdm
import sys

sys.path.append("../../")
from elo.elo import StandingsTable

players = [
    RandomPlayer,
    MCTSPlayer,
    AlphaBetaPlayer,
    # GreedyPlayoutsPlayer,
    VictoryPointPlayer,
    WeightedRandomPlayer,
    # ValueFunctionPlayer,
]
games_per_pair = 10

player_names = [p.__name__ for p in players]
table = StandingsTable(player_names, start_elo=1000)


def play_1v1_tournament(players, games_per_pair, play_game):
    tournament_results = []
    for player1 in players:
        results = play_matches(player1, players, games_per_pair, play_game)
        tournament_results.extend(results)
    tournament_results = pd.DataFrame(
        tournament_results, columns=["player1", "player2", "result"]
    )
    return tournament_results


def play_matches(player1, players, games_per_pair, play_game):
    results = []
    for opponent in players:
        if opponent != player1:
            for _ in range(games_per_pair // 2):
                print(f"Playing {player1.__name__} vs {opponent.__name__} game {_+1}")
                result = play_game(player1, opponent)
                results.append((player1.__name__, opponent.__name__, result))

    for opponent in players:
        if opponent != player1:
            for _ in range(games_per_pair // 2):
                print(f"Playing {opponent.__name__} vs {player1.__name__} game {_+1}")
                result = play_game(opponent, player1)
                results.append(
                    (
                        player_names[players.index(opponent)],
                        player_names[players.index(player1)],
                        result,
                    )
                )
    table.add_results_from_array(results)
    print(table.bayes_elo())
    return results

In [ ]:
import pickle

print(table.bayes_elo())
print(table.get_win_table())
print(table.get_draw_table())
file = "catan_1v1_tournament_results.pkl"
pickle.dump(table, open(file, "wb"))

/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/catan/../../elo/elo.py:7: RuntimeWarning: overflow encountered in power
  return 1.0 / (1 + np.power(10, D / 400))
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/catan/../../elo/elo.py:14: RuntimeWarning: divide by zero encountered in log
  l += winTable[i][j] * np.log(f(elos[i] - elos[j] - eloAdvantage + eloDraw))
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/catan/../../elo/elo.py:15: RuntimeWarning: divide by zero encountered in log
  +0.5 * drawTable[i][j] * np.log(
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/catan/../../elo/elo.py:15: RuntimeWarning: invalid value encountered in scalar multiply
  +0.5 * drawTable[i][j] * np.log(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Users/jonathanlamontange-krat

{'Elo table':                        Elo Games  Score Draws
RandomPlayer           867   100  0.215  0.01
MCTSPlayer             925   100   0.34   0.0
AlphaBetaPlayer       1167   100   0.86   0.0
VictoryPointPlayer     890   100  0.265  0.01
WeightedRandomPlayer   948   100   0.39   0.0
ValueFunctionPlayer   1199   100   0.93   0.0, 'eloAdvantage': -694.359820088658, 'eloDraw': 1694.341294605215}
[[ 0.  7.  0.  6.  8.  0.]
 [13.  0.  0. 13.  8.  0.]
 [20. 20.  0. 20. 20.  6.]
 [13.  7.  0.  0.  6.  0.]
 [12. 12.  0. 14.  0.  1.]
 [20. 20. 14. 20. 19.  0.]]
[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/catan/../../elo/elo.py:15: RuntimeWarning: invalid value encountered in log
  +0.5 * drawTable[i][j] * np.log(


In [12]:
from catanatron import Game, RandomPlayer, Color
from catanatron.players.mcts import MCTSPlayer
from catanatron.players.minimax import AlphaBetaPlayer
from catanatron.players.playouts import GreedyPlayoutsPlayer
from catanatron.players.search import VictoryPointPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.value import ValueFunctionPlayer

table.add_player(GreedyPlayoutsPlayer.__name__)
players.append(GreedyPlayoutsPlayer) if GreedyPlayoutsPlayer not in players else None

play_matches(GreedyPlayoutsPlayer, players, games_per_pair * 2, play_game)

Playing GreedyPlayoutsPlayer vs RandomPlayer game 1
Greedy took 67.27976489067078 secs to decide 54 at 25 per action
Greedy took 3.962010145187378 secs to decide 3 at 25 per action
Greedy took 50.51732397079468 secs to decide 44 at 25 per action
Greedy took 3.214242935180664 secs to decide 3 at 25 per action
Greedy took 5.8630030155181885 secs to decide 5 at 25 per action
Greedy took 12.508062839508057 secs to decide 12 at 25 per action
Greedy took 13.31558871269226 secs to decide 12 at 25 per action
Greedy took 14.557521104812622 secs to decide 16 at 25 per action
Greedy took 14.153875827789307 secs to decide 16 at 25 per action
Greedy took 4.925137996673584 secs to decide 5 at 25 per action
Greedy took 11.22763705253601 secs to decide 12 at 25 per action
Greedy took 10.637969970703125 secs to decide 12 at 25 per action
Greedy took 14.344194173812866 secs to decide 12 at 25 per action
Greedy took 13.968308925628662 secs to decide 12 at 25 per action
Greedy took 20.072431087493896 secs

Process SpawnPoolWorker-8987:
Process SpawnPoolWorker-8989:
Process SpawnPoolWorker-8991:
Process SpawnPoolWorker-8990:
Process SpawnPoolWorker-8992:
Process SpawnPoolWorker-8988:
Process SpawnPoolWorker-8985:
Process SpawnPoolWorker-8986:


KeyboardInterrupt: 

In [9]:
results = play_1v1_tournament(players, games_per_pair, play_game)


# table.add_results_from_dataframe(results)  # Adding multiple results
print(table.bayes_elo())

Playing RandomPlayer vs MCTSPlayer game 1


KeyboardInterrupt: 

In [ ]:
# --- Catanatron Game Logic and Constants (from original code) ---
from catanatron.models.player import Color
from catanatron.models.map import BASE_MAP_TEMPLATE, NUM_NODES, LandTile
from catanatron.models.enums import RESOURCES, Action, ActionType
from catanatron.models.board import get_edges

BASE_TOPOLOGY = BASE_MAP_TEMPLATE.topology
TILE_COORDINATES = [x for x, y in BASE_TOPOLOGY.items() if y == LandTile]
ACTIONS_ARRAY = [
    (ActionType.ROLL, None),
    *[(ActionType.MOVE_ROBBER, tile) for tile in TILE_COORDINATES],
    (ActionType.DISCARD, None),
    *[(ActionType.BUILD_ROAD, tuple(sorted(edge))) for edge in get_edges()],
    *[(ActionType.BUILD_SETTLEMENT, node_id) for node_id in range(NUM_NODES)],
    *[(ActionType.BUILD_CITY, node_id) for node_id in range(NUM_NODES)],
    (ActionType.BUY_DEVELOPMENT_CARD, None),
    (ActionType.PLAY_KNIGHT_CARD, None),
    *[
        (ActionType.PLAY_YEAR_OF_PLENTY, (first_card, RESOURCES[j]))
        for i, first_card in enumerate(RESOURCES)
        for j in range(i, len(RESOURCES))
    ],
    *[(ActionType.PLAY_YEAR_OF_PLENTY, (first_card,)) for first_card in RESOURCES],
    (ActionType.PLAY_ROAD_BUILDING, None),
    *[(ActionType.PLAY_MONOPOLY, r) for r in RESOURCES],
    *[
        (ActionType.MARITIME_TRADE, tuple(4 * [i] + [j]))
        for i in RESOURCES
        for j in RESOURCES
        if i != j
    ],
    *[
        (ActionType.MARITIME_TRADE, tuple(3 * [i] + [None, j]))
        for i in RESOURCES
        for j in RESOURCES
        if i != j
    ],
    *[
        (ActionType.MARITIME_TRADE, tuple(2 * [i] + [None, None, j]))
        for i in RESOURCES
        for j in RESOURCES
        if i != j
    ],
    (ActionType.END_TURN, None),
]
ACTION_SPACE_SIZE = len(ACTIONS_ARRAY)


def normalize_action(action):
    # (Function implementation from the original code)
    normalized = action
    if normalized.action_type == ActionType.ROLL:
        return Action(action.color, action.action_type, None)
    elif normalized.action_type == ActionType.MOVE_ROBBER:
        return Action(action.color, action.action_type, action.value[0])
    elif normalized.action_type == ActionType.BUILD_ROAD:
        return Action(action.color, action.action_type, tuple(sorted(action.value)))
    elif normalized.action_type == ActionType.BUY_DEVELOPMENT_CARD:
        return Action(action.color, action.action_type, None)
    elif normalized.action_type == ActionType.DISCARD:
        return Action(action.color, action.action_type, None)
    return normalized


def to_action_space(action):
    normalized = normalize_action(action)
    return ACTIONS_ARRAY.index((normalized.action_type, normalized.value))


def from_action_space(action_int, playable_actions):
    (action_type, value) = ACTIONS_ARRAY[action_int]
    for action in playable_actions:
        normalized = normalize_action(action)
        if normalized.action_type == action_type and normalized.value == value:
            return action
    raise ValueError(f"Action {action_int} not found in playable_actions")


HIGH = 19 * 5


class CatanPlayerWrapper:
    def __init__(self, player_class, color):
        self.player = player_class(color)
        self.model_name = player_class.__name__

    def predict(self, observation, info, env=None):
        return observation, info, env

    def select_actions(self, prediction, info):
        game = prediction[2].game
        action = self.player.decide(game, game.state.playable_actions)
        # go from catan action to an int
        action_int = to_action_space(action)
        return action_int

In [14]:
# Test a petting zoo environment to see if it has all the functions and attributes needed
from custom_gym_envs.envs.catan import (
    env as catan_env,
    CatanAECEnv,
)
from utils import process_petting_zoo_obs

env = catan_env(
    num_players=2,
    map_type="BASE",
    vps_to_win=10,
    representation="vector",
    invalid_action_reward=-1,
)


# test reset
env.reset()

# Get initial state for first agent
state, reward, termination, truncation, info = env.last()
print(state, info)


ab_player = CatanPlayerWrapper(AlphaBetaPlayer, Color.RED)
prediction = ab_player.predict(state, info, env)
action = ab_player.select_actions(prediction, info)
print(action)

{'observation': array([19.        , 25.        , 19.        , 19.        , 19.        ,
       19.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0